# Supervised learning

Supervised learning covers models that learn from data labelled with the desired solution by a knowledgable external entitiy. In this notebook, the basic concepts and algorithms for supervised learning will be reviewed.

In [1]:
# Add local modules
%load_ext autoreload
%autoreload 2

import sys
import os

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

## Gradient Descent

To learn a model needs to optimize a `cost or loss function` (e.g. **MSE** for regression, **Cross Entropy** `H(p, q)= −Σp(x)log(q(x))` for classification). This optimization is done using Gradient Descent, which finds an optimal (normally a minimum) value by measuring the cost function and its gradient every iteration and moving in the descending direction with a `learning_rate` size step. If this size is too small, the optimization will be slower and if it's too big, it will lead to an unstable process that may not converge to an optimal value.

Depending on the data used to iterate, there are three different versions.
- **Stochastic (SGD)**: it takes one sample every step, it's fast but more unstable   
- **Batch**: it takes the whole training set, it's slow and can lead to overfitting 
- **Mini-Batch**: it takes a random sample every iteration, it's a balanced between SGD and Batch GD. It performs better in GPUs than SGD, as they are optimized for matrix operations.

To remember how gradient descend works, we will build a linear regression model in PyTorch. First, we will use the SGD optimizers from the library and then we will integrate the loss function *manually*.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

N = 64
A = 2
b = 1
error = 0.1

x = Variable(torch.randn(N, 1), requires_grad=False)
y = A*x + b + Variable(torch.randn(N, 1) * error)

A = Variable(torch.randn(1, 1), requires_grad=True)
B = Variable(torch.randn(1), requires_grad=True)

learning_rate = 5e-2

def loss_MSE(predictions, y):
    # same as using nn.MSELoss
    squared_diffs = (predictions - y)**2
    return squared_diffs.mean()

optimizer = optim.SGD([A, B], lr=learning_rate)

for t in range(1000):
    optimizer.zero_grad()
    y_pred = torch.matmul(x, A) + B
    loss = loss_MSE(y_pred, y)
    loss.backward()
    optimizer.step()
    if t%100 == 0:
        print(f'Iteration {t} with loss {loss}:')
        print(f'Y = {A[0, 0]}*x + {B[0]}')

Iteration 0 with loss 12.192981719970703:
Y = 0.19980499148368835*x + -1.1746792793273926
Iteration 100 with loss 0.010363358072936535:
Y = 1.977488398551941*x + 1.010973334312439
Iteration 200 with loss 0.010363351553678513:
Y = 1.9774489402770996*x + 1.0110245943069458
Iteration 300 with loss 0.010363351553678513:
Y = 1.9774489402770996*x + 1.0110245943069458
Iteration 400 with loss 0.010363351553678513:
Y = 1.9774489402770996*x + 1.0110245943069458
Iteration 500 with loss 0.010363351553678513:
Y = 1.9774489402770996*x + 1.0110245943069458
Iteration 600 with loss 0.010363351553678513:
Y = 1.9774489402770996*x + 1.0110245943069458
Iteration 700 with loss 0.010363351553678513:
Y = 1.9774489402770996*x + 1.0110245943069458
Iteration 800 with loss 0.010363351553678513:
Y = 1.9774489402770996*x + 1.0110245943069458
Iteration 900 with loss 0.010363351553678513:
Y = 1.9774489402770996*x + 1.0110245943069458


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

N = 64
A = 2
b = 1
error = 0.1

x = Variable(torch.randn(N, 1), requires_grad=False)
y = A*x + b + Variable(torch.randn(N, 1) * error)

A = Variable(torch.randn(1, 1), requires_grad=True)
B = Variable(torch.randn(1), requires_grad=True)

learning_rate = 5e-2

def GD(x, y, predictions, A, B):
    # Derivate of MSE = (predictions - y)**2/N 
    # dMSE = 2*(predictions - y)/N
    dloss = 2*(predictions - y)/N
    dloss_dw = dloss * x
    dloss_db = dloss
    return torch.stack([dloss_dw.sum(), dloss_db.sum()])

def loss_MSE(predictions, y):
    # same as using nn.MSELoss
    squared_diffs = (predictions - y)**2
    return squared_diffs.mean()

for t in range(1000):
    y_pred = torch.matmul(x, A) + B
    loss = loss_MSE(y_pred, y)
    gradients = GD(x, y, y_pred, A, B)
    A = A - learning_rate * gradients[0]
    B = B - learning_rate * gradients[1]
    if t%100 == 0:
        print(f'Iteration {t} with loss {loss}:')
        print(f'Y = {A[0, 0]}*x + {B[0]}')


Iteration 0 with loss 2.614588737487793:
Y = 0.4222372770309448*x + 0.24920809268951416
Iteration 100 with loss 0.008230608887970448:
Y = 1.9936408996582031*x + 0.9955790638923645
Iteration 200 with loss 0.008229944854974747:
Y = 1.9943785667419434*x + 0.9960685968399048
Iteration 300 with loss 0.008229944854974747:
Y = 1.9943785667419434*x + 0.9960685968399048
Iteration 400 with loss 0.008229944854974747:
Y = 1.9943785667419434*x + 0.9960685968399048
Iteration 500 with loss 0.008229944854974747:
Y = 1.9943785667419434*x + 0.9960685968399048
Iteration 600 with loss 0.008229944854974747:
Y = 1.9943785667419434*x + 0.9960685968399048
Iteration 700 with loss 0.008229944854974747:
Y = 1.9943785667419434*x + 0.9960685968399048
Iteration 800 with loss 0.008229944854974747:
Y = 1.9943785667419434*x + 0.9960685968399048
Iteration 900 with loss 0.008229944854974747:
Y = 1.9943785667419434*x + 0.9960685968399048


## Linear Models

Linear models intent to find a predicted value $\hat{y}$ as a linear combination of weights `w` and features `x`.

The simple linear model is the Ordinary Least Squares [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression), which with regularization parameters in the loss function becomes: 

- [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Lasso): L1 regularization which performs feature selection
- [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge): L2 regularization which reduces the weights of features

For classification, the most used, simple but powerful algortihm is the [Logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression) or `Logit regression` as it uses the logit or `sigmoid function`. 

For `Multinomial Logistic Regression`, the `Softmax` is applied. With a higher order polynomial in the hypothesis function, it can classifiy non-linear data.




In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression

N = 64
A = 2
b = 1
error = 0.1

x = np.random.rand(N, 1)
y = A*x + b + np.random.rand(N, 1) * error

model = LinearRegression().fit(x, y)
print(f'Y = {model.coef_[0, 0]}*x + {model.intercept_[0]}')

Y = 1.9980731092605344*x + 1.0558991996785314


## Support Vector Machines


This method tries to find an optimal
hyperplane that maximizes the margin between data from different classes. The margin is a `Soft Margin` found by minimizing missclassified datapoints. SVM are very powerful when used with kernels (`polynomial`, `Radial Basis Function (RBF)`, `sigmoid tanh`) to classify non-linear datasets getting complicated decision functions. For this, SVM applies the `Kernel Trick`, which calculates the relation of the data in higher dimensions without really transforming the data.

SVM can be used for classification, regression, and even
outlier detection problems.

## Naive Bayes

Based on the Bayes theorem `p(y|x1,..,xn)=p(y)p(x1,..,xn|y)/p(x)`. It is called *naive* as it assumes features are conditionally independent `p(x1|y, x1,..,xn)=p(x1|y)`. `p(y)` and `p(x1|y),...p(xn|y)` are estimated using Maximum A Posteriori (MAP). MAP estimates a distribution of the data based on the observed values.

Depending on the data distribution, different decision rules can be applied. It performs well for document classification and spam filtering.

## Decission Trees

Non-parametrics methods used for classification and regressian. Trees can be visualized, work well with numerical and categorical data, but tends to overfit. Thanks to their simplicity, they are used for ensambling methods. 
One of the most commin decission trees algorithm is the **Classification And Regression Tree (CART)**. It splits the training set in two subsets using a feature `k` and a threshold `tk` so that the data in each group is as pure as possible. 

To measure the quality, it can use:
- `Gini` -> `1 − Σp(i,k)^2` for each class `k` in the node `i` 
- `Entropy`-> `−Σp(i,k)*log(p(i,k))` for each class `k` in the node `i`

Both are zero if all datapoints belong to the same class.

## Ensambling learning

### [Bagging: Bootstrap AGGregatING](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier)

Generates multiple training datasets using random sampling with replacement and trains in parallel different models which will vote to do a final prediction. Bagging helps to avoid overfitting.

[Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) is a bagging method that randomized the features creating more diverse trees.


### Boosting

It starts creating a simple model that can get good results for easy samples, and it gives bigger weights to samples that are difficult to classify. In the next iteration, it will create models focusing on those samples with higher weights. In that way, it optimizes to get higher accuracies. 

[AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) is the classical example of Boosting. AdaBoost predicts the class based on weighted votes, as each estimator (tree) has a weight depending on its accuracy. [Gradient Tree Boosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier) is similar, but uses the residuals errors from the last predictor and supports subsampling or `Stochastic Gradient Boosting`. With Stochastic Gradient Boosting a subsample, whose size is controlled with a parameter `subsample`, is taken to train each tree allowing `out-of-bag error` calcualtion.



# Hyperparameters Tuning or Hyperparameter Optimization (HPO)

Find the parameters of the model that best fit the data maximazing a defined metric. It helps also to find in which moment the model will start overfitting, by comparing the performance between the `training` and `validation` set.

It is normally perfomed using **Cross-Validation** which requires the data being split in three parts: `training`, `test` and `validation`, using the last one to optimize parameters.
If data is scarce, **K-fold Cross-Validation** is recommended. It randomly splits the training dataset into K folds, then it trains the model `K` times with the data in `K-1` folds and evaluate in the one left.

## Grid-Search

`GridSearchCV` in scikit-learn.

Evaluation of different combination of hyperparameters values. Results can be observed with heatmaps or 3D scatter plots where the selected metrics are plotted against the hyperparameters values.

## Randomized Search

`RandomizedSearchCV` in scikit-learn.

Evaluates n iterations with random hyperparameters values. To find the parameters that affect more the perfomance, an ANOVA can show the ones which explain more the performance variance.

## Bayesian Search

It considers past results to create a probabilistic model `p(metric|hyperparameter)` that is used to select the most promising values to be tested in the next iteration. It follos a Bayesian approach, where a more accurate distribution is found with more data. With a good representation, it can in the end choose the optimal value for each hyperparameter. 


# Metrics

## Regression

- Root-mean-square error (RMSE): perferred metric for regression problems as it penalizes higher errors even though this makes it more sensitive to outliers. It's a non-negative value, a value of zero indicates a perfect fit
- Mean Absolute Error (MAE): average of absolute errors
- Mean absolute percentage error (MAPE) or mean absolute percentage deviation (MAPD): average of absolute differences (actual - predictions) divided by the actual values


## Classification

### Confusion matrix

Matrix with predictions (rows) vs actual values (columns). **False Negatives (FN)** indicates the probability of **type II error** (Fail to accept a true hypothesis) and **False Positives (FP)** of **type I error** (Fail to reject a false hypothesis).  From there, we can infer important metrics:

- Recall or Sensitivity or True Positive Rate (TPR): It's the probability of detection -> `TP/(TP + FN)`
- Precision: It's the rate of positives among the predicted as positives -> `TP/(TP + FP)`
- F1: It's the harmonic mean of Recall and Precision -> `2*Precision*Recall/(Precision + Recall)`
- Specificity: It's the probability of detect negatives -> `TN/(TN + FP)` = 1 - FPR (False Positive rate) -> `FP/(FP + TN)`


### Receiver operatir ROC 

Measures the ability of a binary classifier. It's calculated estimating the **TPR** (Y-axis) and **FPR** (X-axis) for different discrimination thresholds. The **Area Under the Curve (AUC)** of the ROC is the probability of classifying a randomly chosen positive instance higher than a randomly chosen negative one.







## Recommender Systems
 
Systemas created to predict users interests based on their previous behavior (e.g. purchase history, ratings, reviews, browsing history) to recommend other products to buy or help them to search. For new users or products, these systems will need some input data (users interests, items details) and a set of rules to start giving recommendations.

- **Content-based systems**: or `personality-based approach` based on characteristic information of the products.
- **Collaborative filtering systems**: use user-item interactions, resulting in recommendation of that type `users who bought X also bought Y`. They can be `memory-based` (clusters of users or items) or `model-based` (e.g. Item2Vec). As large data is required for them to work, they suffer of the `cold start` problem.
- **Hybrid systems**: they combine content and interactions to overcome the shortcomings of each type

To measure their performance, MSE, precision or recall can be used. However, online evaluation is needed for models in production. For that, `A/B tesing` comparing `conversion rate` or `click-through rate` is recommended.



**References**

- GÉRON, Aurélien. Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems. O'Reilly Media, 2019.
- CHOLLET, Francois. Deep Learning with Python and Keras. Manning Publications, 2018.
- [Scikit-Learn User Guide](https://scikit-learn.org/stable/user_guide.html)
